This notebook is for me to play around with different concepts. It is quite messy and is not intended for use in the final application.

In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import argparse
from sklearn.ensemble import RandomForestClassifier
from time import perf_counter
from sklearn import tree

from image_select import *

In [2]:
original = cv2.imread('train_img/t1.jpg')

cv2.imshow('t1', original)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
get_training('train_img/t1.jpg')

Obtaining plant parts...
Obtaining background parts...


(array([[220, 252, 233, ..., 247, 116, 141],
        [224, 255, 237, ..., 250, 117, 141],
        [216, 255, 238, ..., 250, 116, 145],
        ...,
        [123, 188, 166, ..., 187, 109, 158],
        [146, 210, 188, ..., 207, 110, 157],
        [159, 224, 199, ..., 219, 109, 157]], dtype=uint8),
 array([[ 71, 112, 137, ..., 124, 132, 154],
        [ 74, 114, 139, ..., 126, 132, 154],
        [ 81, 121, 146, ..., 133, 132, 154],
        ...,
        [ 95, 142, 164, ..., 153, 129, 156],
        [ 73, 120, 142, ..., 131, 130, 157],
        [ 93, 140, 162, ..., 151, 130, 156]], dtype=uint8))

In [67]:
plant, back = get_training('train_img/t1.jpg')
X_train, y_train = prepare_training(plant, back)

Obtaining plant parts...
Obtaining background parts...


In [68]:
start = perf_counter()
clf = RandomForestClassifier(n_estimators=100)
clf = clf.fit(X_train, y_train)
end = perf_counter()
print(end-start)

1.8555887921379508


In [69]:
# Imported images are stored in a numpy array in B-G-R order.

img_BGR = cv2.imread('train_img/t1.jpg')

# Calculate also the image data in the HSV and LAB colour spaces
img_HSV = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2HSV)
img_LAB = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2LAB)

# concatenate the 3 images to create the input data for the model.
img_final = np.concatenate((img_BGR, img_HSV, img_LAB), axis=2).reshape((-1, 9))

In [70]:
start = perf_counter()
test_pred = clf.predict(img_final)
end = perf_counter()
print(end-start)

test_pred = test_pred.reshape((len(test_pred),1))
test_pred = np.concatenate((test_pred, test_pred, test_pred), axis=1).reshape((img_BGR.shape[0], img_BGR.shape[1], 3))

2.462537905211093


In [71]:
new_image = np.multiply(img_BGR, test_pred)

resized = cv2.resize(new_image, (0,0), fx=0.5, fy=0.5)

cv2.imshow('t1', new_image.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()

Try some erosion and dilation (opening)

In [39]:
# Imported images are stored in a numpy array in B-G-R order.

img_BGR = cv2.imread('train_img/t1.jpg')

# Calculate also the image data in the HSV and LAB colour spaces
img_HSV = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2HSV)
img_LAB = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2LAB)

# concatenate the 3 images to create the input data for the model.
img_final = np.concatenate((img_BGR, img_HSV, img_LAB), axis=2).reshape((-1, 9))

start = perf_counter()
test_pred = clf.predict(img_final)
end = perf_counter()
print(end-start)

test_pred = test_pred.reshape((len(test_pred),1))
test_pred = np.concatenate((test_pred, test_pred, test_pred), axis=1).reshape((img_BGR.shape[0], img_BGR.shape[1], 3))

new_image_1 = np.multiply(img_BGR, test_pred)

cv2.imshow('t1', new_image_1.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()

3.8020472797950333


In [43]:
kernel = np.ones((4,4),np.uint8)
opening = cv2.morphologyEx(test_pred, cv2.MORPH_OPEN, kernel)
new_image_2 = np.multiply(img_BGR, opening)

cv2.imshow('t1', new_image_2.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [72]:
kernel = np.ones((4,4),np.uint8)

erosion = cv2.erode(test_pred,kernel,iterations = 1)
new_image_3 = np.multiply(img_BGR, erosion).astype(np.uint8)

cv2.imshow('t1', new_image_3.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()

We can erode the image so that all that is left is definitely plant, then use that data to re-train the model. This will increase the training set size and hopefully improve results.

Here we will only all pixels that are all zeros.

In [73]:
mask = np.all(new_image_3 == 0, axis=2)
plant_HSV = cv2.cvtColor(new_image_3, cv2.COLOR_BGR2HSV)
plant_LAB = cv2.cvtColor(new_image_3, cv2.COLOR_BGR2LAB)
plant_new = np.concatenate((new_image_3, plant_HSV, plant_LAB), axis=2)

new_plant_data = plant_new[~mask]

In [74]:
X_train, y_train = prepare_training(new_plant_data, back)

start = perf_counter()
clf = RandomForestClassifier(n_estimators=100)
clf = clf.fit(X_train, y_train)
end = perf_counter()
print(end-start)

img_BGR = cv2.imread('train_img/t1.jpg')
img_HSV = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2HSV)
img_LAB = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2LAB)
img_final = np.concatenate((img_BGR, img_HSV, img_LAB), axis=2).reshape((-1, 9))

start = perf_counter()
test_pred = clf.predict(img_final)
end = perf_counter()
print(end-start)

test_pred = test_pred.reshape((len(test_pred),1))
test_pred = np.concatenate((test_pred, test_pred, test_pred), axis=1).reshape((img_BGR.shape[0], img_BGR.shape[1], 3))

3.965928876227281
2.41469521871295


In [75]:
new_image = np.multiply(img_BGR, test_pred)

resized = cv2.resize(new_image, (0,0), fx=0.5, fy=0.5)

cv2.imshow('t1', new_image.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()

It seems that adding erosion does not improve the training data set, which is strange.